Just a reminder that the implementation for ACE is written in Julia and there is quite a lot of stuf to install. If you are having issues with the julia environment, you can install the whole thing yourself by uncommenting the lines in the cell below #pkg"registry add https://github.com/JuliaRegistries/General"; pkg"registry add https://github.com/JuliaMolSim/MolSim.git"; pkg"add ACE1pack, ACE1, JuLIP, IPFitting, ASE"


In [8]:
using Pkg; Pkg.activate(".");#pkg"registry add https://github.com/JuliaRegistries/General"; pkg"registry add https://github.com/JuliaMolSim/MolSim.git"; pkg"add ACE1pack, ACE1, JuLIP, IPFitting, ASE"
using ACE1pack
using DelimitedFiles

  Activating project at `~/gbcompare/ace_scripts`


In [7]:
my_xyzs = readdir("xyz_data",join=true)

#The sorted array is not crucial because we are naming our files, but I am doing it anyway.
my_sorted_xyzs = sort(my_xyzs)

basis = ace_basis(species = [:Al],
      N = 3,        # correlation order = body-order - 1
      maxdeg = 12,  # polynomial degree
      r0 = 2.86,     # estimate for NN distance
      rin = 0.1,
      rcut = 3.25,
      pin = 2)

function get_descriptor(dataset)
    # compute average descriptor for each structure
    descriptor = zeros(length(dataset[1]),length(basis))
    for i in 1:length(dataset[1])
        descriptor[i,:]=site_energy(basis, dataset[1], i) / length(dataset[1])
    end
return descriptor
end

directory = 'ace_txt_data'

if not os.path.exists(directory):
    os.makedirs(directory)
    print("Directory created:", directory)
else:
    print("Directory already exists:", directory)

for f in my_xyzs[1:length(my_xyzs)]
    dataset = JuLIP.read_extxyz(f);
    data = get_descriptor(dataset)
    split_f = split(split(f,"/")[end],".")
    writedlm("ace_txt_data/" * split_f[1]*"."*split_f[2] * ".txt",data)
end